<a href="https://colab.research.google.com/github/ssudhakar165/colab/blob/main/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib
from PIL import Image

import torch
import torch.nn as nn
from torch.nn import Linear, Sequential, ReLU, Sigmoid

from torch.optim import adam
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet34


Using matplotlib backend: agg


In [81]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/emergency.zip'

Archive:  /content/drive/MyDrive/emergency.zip
   creating: Dataset/
  inflating: Dataset/emergency_classification.csv  
   creating: Dataset/images/
  inflating: Dataset/images/0.jpg    
  inflating: Dataset/images/10.jpg   
  inflating: Dataset/images/1002.jpg  
  inflating: Dataset/images/1006.jpg  
  inflating: Dataset/images/1007.jpg  
  inflating: Dataset/images/1009.jpg  
  inflating: Dataset/images/1013.jpg  
  inflating: Dataset/images/1014.jpg  
  inflating: Dataset/images/1016.jpg  
  inflating: Dataset/images/1019.jpg  
  inflating: Dataset/images/102.jpg  
  inflating: Dataset/images/1020.jpg  
  inflating: Dataset/images/1022.jpg  
  inflating: Dataset/images/1023.jpg  
  inflating: Dataset/images/1026.jpg  
  inflating: Dataset/images/1031.jpg  
  inflating: Dataset/images/1035.jpg  
  inflating: Dataset/images/1036.jpg  
  inflating: Dataset/images/1037.jpg  
  inflating: Dataset/images/104.jpg  
  inflating: Dataset/images/1040.jpg  
  inflating: Dataset/images/1041.jp

In [82]:
normalize= transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

preprocess= transforms.Compose([transforms.ToTensor(), normalize,])

In [83]:
class dat(Dataset):
  def __init__ (self, csv_path, img_dir, transform):
    df= pd.read_csv(csv_path)
    self.img_dir= img_dir
    self.csv_path= csv_path
    self.transform= transform
    self.img_names= df.image_names.values
    self.y= df["emergency_or_not"].values

  def __getitem__(self, index):
    img= Image.open(self.img_dir + self.img_names[index])
    if self.transform is not None:
       img = self.transform(img)
        
    label = self.y[index]
    return img, label

    # defining the len function
  def __len__(self):
    return self.y.shape[0]



In [84]:
train_dataset= dat(csv_path= '/content/Dataset/emergency_classification.csv', img_dir='/content/Dataset/images/', transform=preprocess)

In [85]:
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_data, test_data= torch.utils.data.random_split(train_dataset, [train_size, test_size])

In [86]:
train_loader= DataLoader(train_data, batch_size=32)
test_loader= DataLoader(test_data, batch_size=32)

In [87]:
model= resnet34(pretrained= True)

In [90]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [91]:
model1= Sequential( Linear(in_features=1000, out_features=50),
                    ReLU(),
                    Linear(50, 1),
                    Sigmoid())

In [92]:
model1

Sequential(
  (0): Linear(in_features=1000, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=1, bias=True)
  (3): Sigmoid()
)

In [93]:
final= Sequential(model, model1)

In [94]:
final

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [95]:
for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
  break


In [96]:
input = batch_X
output= final(input)
output.shape

torch.Size([32, 1])

In [ ]:
optimizer = torch.optim.Adam(final.parameters())
criterion = nn.BCELoss()
if torch.cuda.is_available():
    final_model = final.cuda()
    criterion = criterion.cuda()

In [ ]:

def binary_accuracy(preds, y):
  
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)

    #no. of correctly classified    
    correct = (rounded_preds == y).float()

    #compute accuracy 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(net, train_loader):
    for epoch in range (10): # no. of epochs
        training_loss = 0
        epoch_acc=0
        best_acc=0
        for batch_idx, (batch_x, batch_y) in (enumerate(train_loader)):
          if torch.cuda.is_available():
             batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
          net.train()
          optimizer.zero_grad()
          outputs = net(batch_x)
          outputs= outputs.squeeze()
          loss = criterion(outputs, batch_y.float())
          acc = binary_accuracy(outputs, batch_y)  
          ##output= outputs.detach().numpy()
          loss.backward()
          optimizer.step()
          training_loss += loss.item()
          epoch_acc  = epoch_acc  + acc.item()
            ##print(epoch_acc)
        training_loss = np.average(training_loss)
        if best_acc <= acc :
           best_acc = acc
           torch.save(final_model.state_dict(), 'save_weights.pth')
           print('Best model saved')
        print( epoch, training_loss,acc)

In [ ]:
train(final, train_loader)

Best model saved
0 20.99511980265379 tensor(0.8800, device='cuda:0')
Best model saved
1 13.654219064861536 tensor(0.9200, device='cuda:0')
Best model saved
2 11.039660353213549 tensor(0.9200, device='cuda:0')
Best model saved
3 8.279482558369637 tensor(0.9600, device='cuda:0')
Best model saved
4 6.250005472218618 tensor(0.9200, device='cuda:0')
Best model saved
5 5.930999840144068 tensor(0.9200, device='cuda:0')
Best model saved
6 5.290134605020285 tensor(1., device='cuda:0')
Best model saved
7 3.5687152764294297 tensor(0.9200, device='cuda:0')
Best model saved
8 3.634626362938434 tensor(0.9600, device='cuda:0')
Best model saved
9 2.810799403639976 tensor(0.9600, device='cuda:0')


In [ ]:
  acc1 = 0
  test_prediction=[]
  for batch_idx, (batch_x, batch_y) in (enumerate(train_loader)): 
    if torch.cuda.is_available():
      batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
    model.eval()
    output = final(batch_x)
    output= output.squeeze()
    acc =  binary_accuracy(output, batch_y) 
  
    acc1=acc1+acc
    test_prediction.append(acc)
  print(acc1/len(train_loader))

tensor(0.9834, device='cuda:0')


In [ ]:
  acc2 = 0
  test=[]
  for batch_idx, (batch_x, batch_y) in (enumerate(test_loader)): 
    if torch.cuda.is_available():
       batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
    model.eval()
    output = final_model(batch_x)
    output= output.squeeze()
    acc =  binary_accuracy(output, batch_y) 
  
    acc2=acc2+acc
    test.append(acc)
  print(acc2/len(test_loader))

tensor(0.9313, device='cuda:0')


**Features Extract**

In [ ]:
train_dataset= dat(csv_path= '/content/Dataset/emergency_classification.csv', img_dir='/content/Dataset/images/', transform=preprocess)

In [ ]:
train_loader= DataLoader(train_dataset, batch_size=32)

In [ ]:
fmodel= resnet34(pretrained=True)
fmodel

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
fmodel.avgpool=nn.Sequential()
fmodel.fc= nn.Sequential()
fmodel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
  break

In [ ]:
batch_X.shape, batch_y.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [ ]:
input= batch_X[:2]
out= fmodel(input)
out.shape

torch.Size([2, 25088])

In [ ]:
features=[]
target=[]
fmodel.eval()

# deactivates autograd
with torch.no_grad():
  for batch_idx, (batch_X, batch_y) in (enumerate(train_loader)):
    feat= fmodel(batch_X)
    feat=feat.data.cpu().numpy()
    features.append(feat)
    target.append(batch_y)
  features= np.concatenate(features, axis=0)
  target= np.concatenate(target, axis=0)



In [ ]:
features.shape, target.shape

((2352, 25088), (2352,))

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest= train_test_split(features, target, test_size=.2, stratify= target, random_state=42)
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((1881, 25088), (1881,), (471, 25088), (471,))

In [ ]:
xtrain= torch.FloatTensor(xtrain)
ytrain= torch.FloatTensor(ytrain)
xtest= torch.FloatTensor(xtest)
ytest= torch.FloatTensor(ytest)

In [ ]:
fet_model= Sequential( Linear(25088,64),
                      ReLU(),
                      Linear(64,1),
                       Sigmoid())

In [ ]:
fet_model

Sequential(
  (0): Linear(in_features=25088, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=1, bias=True)
  (3): Sigmoid()
)

In [ ]:
optimiser=torch.optim.Adam(fet_model.parameters())
criterion= nn.BCELoss()

In [ ]:
def binary(preds, y):
  
    # round predictions to the closest integer
    rounded_preds = torch.round(preds)

    # no. of correctly classified    
    correct = (rounded_preds == y).float()

    # compute accuracy 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
epochs=10

def fet_train(x, y, batch_size):
  for epoch in range(epochs):
    fet_model.train()
    loss1, acc1= 0,0
    indices=torch.randperm(len(x))
    batch=0
    for i in range(0, len(indices), batch_size):
      ind=indices[i:i+batch_size]
      batch_x= x[ind]
      batch_y= y[ind]
      optimiser.zero_grad()
      output=fet_model(batch_x)
      output=output.squeeze()
      loss=criterion(output, batch_y)
      acc= binary(output, batch_y)
      loss.backward()
      optimiser.step()
      loss1=loss1+loss.item()
      acc1= acc1+acc.item()
      batch=batch+1
    print(epoch, loss1/batch, acc1/batch)
  return loss1/batch, acc1/batch

 
    




In [ ]:
fet_train(xtrain, ytrain, 32)

0 0.0019641649238511198 0.9989406779661016
1 0.0019548843333646124 0.9989406779661016
2 0.001947575693382039 0.9989406779661016
3 0.0019400176532422015 0.9989406779661016
4 0.0020354034949362403 0.9989406779661016
5 0.0019290435582168019 0.9989406779661016
6 0.0019243260962018937 0.9989406779661016
7 0.0019192810633531904 0.9989406779661016
8 0.002016907960489211 0.9989406779661016
9 0.001910292339713216 0.9989406779661016


(0.001910292339713216, 0.9989406779661016)

In [ ]:
def fet_test(x, y, batch_size):
    fet_model.eval()
    loss1, acc1= 0,0
    indices=torch.randperm(len(x))
    batch=0
    for i in range(0, len(indices), batch_size):
      ind=indices[i:i+batch_size]
      batch_x= x[ind]
      batch_y= y[ind]
      with torch.no_grad():
         output=fet_model(batch_x)
         output=output.squeeze()
         loss=criterion(output, batch_y)
         acc= binary(output, batch_y)
         loss1=loss1+loss.item()
         acc1= acc1+acc.item()
      batch=batch+1
    return loss1/batch, acc1/batch

In [ ]:
fet_test(xtrain, ytrain, 32)

(0.0019065521769874268, 0.9989406779661016)

In [ ]:
fet_test(xtest,ytest,32)

(0.26334703477720417, 0.9442028999328613)